# TL;DR

This notebook will gather the metrics normally used to evaluate a quota change request. It is meant to automate the research done manually and apply herustics to guide judgement

Enter the namespace of the project you are evaluating.

In [3]:
from array import *
import os
import pandas as pd
import requests
import sqlalchemy as db

In [35]:
namespace='d106d6-dev'
throttling_threshold=9 # as a percentage
thanos_token = os.environ['THANOS_TOKEN']
thanos_url = os.environ['THANOS_URL']

In [79]:
headers = { 'Content-type': 'application/json', 'Authorization': 'Bearer %s' % thanos_token } 

def letter_for_grade(grade):
    if grade >= 80:
        result = [grade, 'A']
    elif grade >= 60 and grade < 80:
        result = [grade, 'B']
    elif grade >= 40 and grade < 60:
        result = [grade, 'C']
    elif grade >= 20 and grade < 40:
        result = [grade, 'D']
    else:
        result = [grade, 'F']

    return result

def fetch_and_process_slack():
  querystring = 'query=sum({__name__=~"namespace_cpu:kube_pod_container_resource_requests:sum|namespace:container_cpu_usage:sum|namespace_cpu:kube_pod_container_resource_limits:sum", namespace=~"%s"} offset 1d) BY (__name__)'
  temp = {}
  p = querystring % namespace

  response = requests.get(thanos_url, headers = headers, params = p)
  if response.status_code != 200:
    quit()

  result_as_json = pd.read_json(response.text)

  try:
    # print(response.text)
    temp[namespace] = {}
    for r in result_as_json.data['result']:
      temp[namespace][r['metric']['__name__']] = round(float(r['value'][1]), 2)
    if len(temp[namespace].keys()) == 0:
      del temp[namespace]
    
  except Exception as e:
    print(e)

  u = temp[namespace]['namespace:container_cpu_usage:sum']
  r = temp[namespace]['namespace_cpu:kube_pod_container_resource_requests:sum']
  l = temp[namespace]['namespace_cpu:kube_pod_container_resource_limits:sum']
  s = round(float(u/r*100), 0)

  return {'usage': u, 'request': r, 'limit': l, 'slack_as_pct': s}

def fetch_and_process_throttling():
  temp = []
  querystring = 'query=avg(container_cpu_cfs_throttled_periods_total / container_cpu_cfs_periods_total {namespace=~"%s", pod=~".*", container_name!="POD", image!=""} * 100 > %x) BY (pod)'
  p = querystring % (namespace, throttling_threshold)

  response = requests.get(thanos_url, headers = headers, params = p)
  if response.status_code != 200:
    quit()

  result_as_json = pd.read_json(response.text)

  try:
    for r in result_as_json.data['result']:
      temp.append({ 'name': r['metric']['pod'], 'value':round(float(r['value'][1]), 0) })

  except Exception as e:
    print(e)

  return temp

# utilization = fetch_and_process_slack()
# throttled = fetch_and_process_throttling()

# metrics  = {'throttled': throttled, 'utilization': utilization}

metrics = { 'throttled': [{'name': 'abc', 'value': 42.1}], 'utilization': {'usage': 0.8, 'request': 3.3, 'limit': 5.0, 'slack_as_pct': 2.1}}


# Evaluation

The following are to be used 
## Current Status
Any project with less than a F or D grade for CPU utilization must book a 30 minute meeting to review the presentation on how to optimize resouce usage on OpenShift.

In [92]:
print('CPU Usage & Throtteling Report\n')
print('CPU usage  : %s' % metrics['utilization']['usage'])
print('CPU request: %s' % metrics['utilization']['request'])
print('CPU limit  : %s' % metrics['utilization']['limit'])
print('CPU slack  : %s' % metrics['utilization']['slack_as_pct'])
print('\nNamespace %s gets a %s for CPU utilization.' % (namespace, letter_for_grade(metrics['utilization']['slack_as_pct'])[1]))
print('\nThe following pods are being throttled \na significant amount:')

for t in metrics['throttled']:
    print('  - %s, time throttled: %f' % (t['name'], t['value']))


CPU Usage & Throtteling Report

CPU usage  : 0.8
CPU request: 3.3
CPU limit  : 5.0
CPU slack  : 2.1

Namespace d106d6-dev gets a F for CPU utilization.

The following pods are being throttled 
a significant amount:
  - abc, time throttled: 42.100000
